## How To Perform an Resonator Spectroscopy Experiment

TODO: Substitute experiment with dsl once QuantumOperations are re-worked to exclude reserve

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see these guides (add links) if you need to create your setup and qubits for the first time. However, you can also run this notebook "as is" using an emulated session. If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this notebook, we will learn how to extend the workflow of resonator spectroscopy to include the sweep of the dc bias provided by a third party instruments. In this tutorial, we will in order do the following:

- Extend a Tunable Transmon functionality to include the property of DC bias
- Learn how to create a new Quantum Operation to sweep the bias how a Tunable Transmon and include it in our standard set of operations
- Learn how to manipulate third-party devices using neartime-callback functions inside Quantum Operations
- Build a new Workflow that exploit the new functionalities

### Imports

You'll start by importing `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [1]:
from __future__ import annotations

import numpy as np
from laboneq.simple import *

from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup.

In [2]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Set the parameter of a Tunable Transmon

First, let's set the relevant qubit parameters for controlling an external DC voltage source. The `TunableTransmon` object already posses two parameters to describe the DC bias of the qubit:
- **dc_slot** to describe to which channel the qubit is connected to
- **dc_voltage_parking** to describe how much voltage the channel should provide.

In [4]:
for n, qubit in enumerate(my_platform.qpu.qubits):
    qubit.parameters.dc_slot = n
    qubit.parameters.dc_voltage_parking = 0

Above we used as convention that the qubit are connected to the channel in order, and we set their initial value of the DC bias to 0 Volt. Let's look at a qubit to see how this object changed:

In [ ]:
print(my_platform.qpu.qubits[2].parameters)

## Create a Quantum Operation to set the DC bias for a qubit

Next, we want to create a Quantum Operations to set the DC bias of a qubit. To do this, we will follow these steps:
- Create a function to set the DC in a third-party instrument
- Register the function to our session with a standard name
- Create a Quantum Operation that uses the function from the session
- Register the new quantum operation in our platform

### Create a function to set the DC bias of qubit

Next, we need to provide a function to set the DC bias of a particular qubit. The exact form of this function will depend on the driver of the device that is to be used, for the purpose of this tutorial, we will just use a mock to show how this is done. Following the prescription provided in the documentation of [Neartime-Callback Functions and 3rd-Party Devices](https://docs.zhinst.com/labone_q_user_manual/concepts/callback_functions/), the first argument of such function should always be the session.

In [6]:
def my_setter(
    session: Session,
    voltage,
    channel,
):
    return f"channel={channel}, voltage={voltage:.1f}"

### Register function in the session and create a matching Quantum Operation

The next two steps will be performed together, we do this to be sure that there is a proper match between the function register and the Quantum Operation that uses it. Thanks to the quantum operation, we can be sure that this function will be used with the correct parameters relevant to a particular qubit each time. Notice how we allow the voltage to be selected optionally by the user. This allow us to override the voltage stored in the qubit parameter if it is needed for particular reasons, for example to sweep it in the context of an experiment.

In [7]:
from laboneq_applications import dsl

func_id = "set_dc_bias"

session.register_neartime_callback(my_setter, func_id)


@my_platform.qpu.qop.register
def set_dc_bias(
    self,
    qubit,
    voltage=None,
):
    # fetch parameters
    ## voltage if not provided
    if voltage is None:
        voltage = qubit.parameters.dc_voltage_parking
    ## channel always provided by the qubit
    channel = qubit.parameters.dc_slot

    # call the function
    dsl.call(func_id, voltage=voltage, channel=channel)

let's convince ourselves that the operation is there by inspecting the source code:

In [ ]:
my_platform.qpu.qop.set_dc_bias.src

## Create a new Workflow using the new Quantum Operation

We are now set to include the new Quantum Operation in a workflow of our choice. For the purpose of this exercise, let's create a simplified version of a pulsed resonator spectroscopy were the DC bias is swept together with the frequency sent to the measure line.

Creating a new `Workflow` requires additional tools from the Applications Library, so let's go ahead and import them.

In [9]:
# Import to build a new workflow
from laboneq_applications import dsl, workflow
from laboneq_applications.tasks import (
    compile_experiment,
    run_experiment,
)

Now let's write the experiment using the dsl. We used a code similar to the `resonator_spectroscopy_amplitude` but simpler. The main change is that now instead of sweeping the amplitude we use our new Quantum Operation for changing the DC bias, and we pass a sweep parameter to it.

In [10]:
@workflow.task
@dsl.qubit_experiment
def create_experiment(
    qpu,
    qubit,
    frequencies,
    voltages,
):
    with dsl.sweep(
        parameter=SweepParameter(f"voltages{qubit.uid}", voltages),
    ) as voltage:
        with dsl.section():
            qpu.qop.set_dc_bias.omit_section(qubit, voltage=voltage)  # set dc bias here
        with dsl.acquire_loop_rt(
            count=100,
            averaging_mode=AveragingMode.SEQUENTIAL,
            acquisition_type=AcquisitionType.SPECTROSCOPY,
        ):
            with dsl.sweep(
                name=f"freq_{qubit.uid}",
                parameter=SweepParameter(f"frequencies_{qubit.uid}", frequencies),
            ) as frequency:
                qpu.qop.set_frequency(qubit, frequency=frequency, readout=True)
                qpu.qop.measure(qubit, dsl.handles.result_handle(qubit.uid))
                qpu.qop.delay(qubit, 1e-6)

Now we are ready to define a new Workflow we the help of standard imported tasks like `compile_experiment` and `run_experiment` together with `create_experiment`.

In [11]:
@workflow.workflow
def res_spec_with_dc_workflow(
    session,
    qpu,
    qubit,
    frequencies,
    voltages,
):
    # create experiment
    exp = create_experiment(
        qpu,
        qubit,
        frequencies=frequencies,
        voltages=voltages,
    )
    # compile it
    compiled_exp = compile_experiment(session, exp)
    # run it
    _result = run_experiment(session, compiled_exp)

#### Run the experiment

We are now good to go! Let's run of workflow with some test voltages. We should expect the printout of our mock function to appear with the voltages we passed

In [ ]:
my_workflow = res_spec_with_dc_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubit=qubit,
    frequencies=np.linspace(6.5e9, 7.5e9, 1001),
    voltages=[0.5, 1.0, 2.0, 3.0],
)

my_results = my_workflow.run()

To verify that the function was run, let's inspect the result object and check the neartime-callbacks

In [ ]:
my_results.tasks["run_experiment"].output.neartime_callbacks["set_dc_bias"]

#### Add task to a workflow

Now that we convinced ourselves that this work, and used the previous experiment to find the optimal parking voltages for each qubit, we can expand this concept and use `Task` to define a calibration for the DC sources. First, let's set some mockup values for the voltage to the qubits in our platform.

In [14]:
voltages = [1, 1.3, 1.5, 1.2, 1.7, 0.9]
for voltage, qubit in zip(voltages, my_platform.qpu.qubits):
    qubit.parameters.dc_parking_voltage = voltage

Next, let's define a `Task` to automatically set the correct DC biases. Some DC sources allow for parallel settings of these values, so let's explore this more general case to exploit this feature. We will prepare a dictionary with parameters starting from the `QPU`

In [15]:
@workflow.task
def set_all_dc(
    qpu,
):
    dc_dict = {}
    for qubit in qpu.qubits:
        dc_dict[qubit.uid] = {
            "channel": qubit.parameters.dc_slot,
            "voltage": qubit.parameters.dc_parking_voltage,
        }

    # mocking the voltage settings
    for key, value in dc_dict.items():
        voltage = value["voltage"]
        channel = value["channel"]
        print(f"voltage {voltage} Volt was set in channel {channel} for qubit {key}")

Now let's assume we want to make sure all values are correctly set before we run another workflow. To do this, we use an existing workflow and we add the above task to perform this calibration

In [16]:
from laboneq_applications.experiments import amplitude_rabi, options


@workflow.workflow
def new_rabi_workflow(
    session,
    qpu,
    qubits,
    amplitudes,
    options: options.TuneUpWorkflowOptions | None = None,
):
    # calibrate dc sources
    set_all_dc(qpu)
    exp = amplitude_rabi.create_experiment(
        qpu,
        qubits,
        amplitudes=amplitudes,
    )
    compiled_exp = compile_experiment(session, exp)
    _result = run_experiment(session, compiled_exp)

Let's check that the printout are there by running the workflow!

In [ ]:
new_workflow = new_rabi_workflow(
    session,
    my_platform.qpu,
    my_platform.qpu.qubits,
    amplitudes=[np.linspace(0.1, 1.0, 11) for n in range(6)],
)

workflow_result = new_workflow.run()